In [1]:
import os
import hashlib
from pathlib import Path
import subprocess

# First, let's explore the assets directory structure
assets_path = "/workspace/shadcn-ui/public/assets/"

print("=== ASSETS DIRECTORY ANALYSIS ===\n")

# Check if assets directory exists
if os.path.exists(assets_path):
    print(f"Assets directory found: {assets_path}")
    
    # List all files in the assets directory
    all_files = []
    for root, dirs, files in os.walk(assets_path):
        for file in files:
            file_path = os.path.join(root, file)
            file_size = os.path.getsize(file_path)
            all_files.append({
                'path': file_path,
                'name': file,
                'size': file_size,
                'extension': os.path.splitext(file)[1].lower()
            })
    
    print(f"\nTotal files found: {len(all_files)}")
    print("\nAll files in assets directory:")
    print("-" * 80)
    
    # Sort files by name for better organization
    all_files.sort(key=lambda x: x['name'].lower())
    
    for file_info in all_files:
        print(f"📁 {file_info['name']:<30} | Size: {file_info['size']:>8} bytes | Ext: {file_info['extension']}")
    
    # Filter image files
    image_extensions = {'.png', '.jpg', '.jpeg', '.gif', '.svg', '.webp', '.bmp', '.ico'}
    image_files = [f for f in all_files if f['extension'] in image_extensions]
    
    print(f"\n=== IMAGE FILES ANALYSIS ===")
    print(f"Image files found: {len(image_files)}")
    print("-" * 80)
    
    for img in image_files:
        print(f"🖼️  {img['name']:<30} | Size: {img['size']:>8} bytes | Type: {img['extension']}")
    
else:
    print(f"Assets directory not found: {assets_path}")

=== ASSETS DIRECTORY ANALYSIS ===

Assets directory found: /workspace/shadcn-ui/public/assets/

Total files found: 14

All files in assets directory:
--------------------------------------------------------------------------------
📁 businessloan.png               | Size:  1042848 bytes | Ext: .png
📁 Carloan.jpg                    | Size:  2188462 bytes | Ext: .jpg
📁 carloan.jpg                    | Size:    97913 bytes | Ext: .jpg
📁 carloan.png                    | Size:  1175290 bytes | Ext: .png
📁 favicon.ico                    | Size:    13833 bytes | Ext: .ico
📁 herobanner.png                 | Size:  2133903 bytes | Ext: .png
📁 homeloan.png                   | Size:  1116592 bytes | Ext: .png
📁 instagram.svg                  | Size:     2460 bytes | Ext: .svg
📁 loanagainstproperty.png        | Size:  1243960 bytes | Ext: .png
📁 logo-alt.png                   | Size:   733014 bytes | Ext: .png
📁 logo.png                       | Size:   294070 bytes | Ext: .png
📁 personalloan.png   

In [2]:
import os
import re
from pathlib import Path

# Search through all component files to find image references
print("=== SEARCHING FOR IMAGE REFERENCES IN CODEBASE ===\n")

# Define the source directory to search
src_path = "/workspace/shadcn-ui/src/"
file_extensions = ['.tsx', '.ts', '.jsx', '.js', '.html', '.css']

# Function to search for image references in a file
def find_image_references(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            
        # Patterns to match image references
        patterns = [
            r'/assets/([^"\'\s]+\.(png|jpg|jpeg|gif|svg|webp|bmp|ico))',  # /assets/image.ext
            r'assets/([^"\'\s]+\.(png|jpg|jpeg|gif|svg|webp|bmp|ico))',   # assets/image.ext
            r'"([^"]*\.(png|jpg|jpeg|gif|svg|webp|bmp|ico))"',           # "image.ext"
            r"'([^']*\.(png|jpg|jpeg|gif|svg|webp|bmp|ico))'",           # 'image.ext'
        ]
        
        found_images = set()
        for pattern in patterns:
            matches = re.findall(pattern, content, re.IGNORECASE)
            for match in matches:
                if isinstance(match, tuple):
                    image_name = match[0] if match[0] else match[1]
                else:
                    image_name = match
                found_images.add(image_name)
        
        return list(found_images)
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return []

# Collect all source files
source_files = []
if os.path.exists(src_path):
    for root, dirs, files in os.walk(src_path):
        for file in files:
            if any(file.endswith(ext) for ext in file_extensions):
                source_files.append(os.path.join(root, file))

print(f"Searching through {len(source_files)} source files...")
print("-" * 80)

# Track all image references found
all_image_references = {}
total_references = 0

for file_path in source_files:
    images = find_image_references(file_path)
    if images:
        relative_path = file_path.replace("/workspace/shadcn-ui/", "")
        all_image_references[relative_path] = images
        total_references += len(images)
        print(f"📄 {relative_path}")
        for img in images:
            print(f"   └── 🖼️  {img}")
        print()

print(f"\n=== SUMMARY OF IMAGE REFERENCES ===")
print(f"Total files with image references: {len(all_image_references)}")
print(f"Total image references found: {total_references}")

# Create a set of all referenced images (normalize names)
referenced_images = set()
for file_images in all_image_references.values():
    for img in file_images:
        # Clean up the image name (remove path prefixes)
        clean_name = img.split('/')[-1] if '/' in img else img
        referenced_images.add(clean_name.lower())

print(f"\nUnique images referenced in code: {len(referenced_images)}")
print("-" * 50)
for img in sorted(referenced_images):
    print(f"✅ {img}")

=== SEARCHING FOR IMAGE REFERENCES IN CODEBASE ===

Searching through 67 source files...
--------------------------------------------------------------------------------
📄 src/components/Testimonials.tsx
   └── 🖼️  testimonial-2.jpg
   └── 🖼️  testimonial-5.jpg
   └── 🖼️  /assets/testimonial-3.jpg
   └── 🖼️  /assets/testimonial-4.jpg
   └── 🖼️  /assets/testimonial-1.jpg
   └── 🖼️  testimonial-3.jpg
   └── 🖼️  /assets/testimonial-5.jpg
   └── 🖼️  /assets/testimonial-2.jpg
   └── 🖼️  testimonial-4.jpg
   └── 🖼️  testimonial-1.jpg

📄 src/pages/About.tsx
   └── 🖼️  /assets/homeloan.png
   └── 🖼️  businessloan.png
   └── 🖼️  logo.png
   └── 🖼️  /assets/logo.png
   └── 🖼️  homeloan.png

📄 src/pages/Contact.tsx
   └── 🖼️  /assets/logo.png
   └── 🖼️  logo.png

📄 src/pages/EMICalculator.tsx
   └── 🖼️  /assets/logo.png
   └── 🖼️  logo.png

📄 src/pages/Index.tsx
   └── 🖼️  logo-alt.png
   └── 🖼️  /assets/businessloan.png
   └── 🖼️  /assets/homeloan.png
   └── 🖼️  loanagainstproperty.png
   └── 🖼️

In [3]:
print("=== COMPREHENSIVE IMAGE CLEANUP REPORT ===\n")

# Images found in assets directory (from previous analysis)
assets_images = {
    'businessloan.png': 1042848,
    'Carloan.jpg': 2188462,  # Capital C
    'carloan.jpg': 97913,    # lowercase c - POTENTIAL DUPLICATE
    'carloan.png': 1175290,  # Different extension - POTENTIAL DUPLICATE
    'favicon.ico': 13833,
    'herobanner.png': 2133903,
    'homeloan.png': 1116592,
    'instagram.svg': 2460,
    'loanagainstproperty.png': 1243960,
    'logo-alt.png': 733014,
    'logo.png': 294070,
    'personalloan.png': 1220082,
    'whatsapp.svg': 1659,
    'youtube.svg': 761
}

# Images referenced in code (from previous analysis)
referenced_images = {
    'businessloan.png', 'carloan.jpg', 'herobanner.png', 'homeloan.png',
    'loanagainstproperty.png', 'logo-alt.png', 'logo.png', 'personalloan.png',
    'testimonial-1.jpg', 'testimonial-2.jpg', 'testimonial-3.jpg', 
    'testimonial-4.jpg', 'testimonial-5.jpg'
}

print("🔍 DUPLICATE IMAGE ANALYSIS")
print("=" * 60)

# Identify car loan duplicates
carloan_files = [name for name in assets_images.keys() if 'carloan' in name.lower()]
print("🚗 Car Loan Image Duplicates Found:")
for file in carloan_files:
    size_mb = assets_images[file] / (1024 * 1024)
    status = "✅ USED" if file.lower() in referenced_images else "❌ UNUSED"
    print(f"   • {file:<20} | {size_mb:.2f} MB | {status}")

print(f"\n📊 USAGE ANALYSIS")
print("=" * 60)

# Categorize images
used_images = []
unused_images = []
missing_images = []

for img_name, size in assets_images.items():
    if img_name.lower() in referenced_images:
        used_images.append((img_name, size))
    else:
        unused_images.append((img_name, size))

# Check for missing images (referenced but not found in assets)
for ref_img in referenced_images:
    if not any(ref_img.lower() == asset.lower() for asset in assets_images.keys()):
        missing_images.append(ref_img)

print(f"✅ IMAGES IN USE ({len(used_images)}):")
for img, size in used_images:
    size_mb = size / (1024 * 1024)
    print(f"   • {img:<25} | {size_mb:>6.2f} MB")

print(f"\n❌ UNUSED IMAGES ({len(unused_images)}):")
total_unused_size = 0
for img, size in unused_images:
    size_mb = size / (1024 * 1024)
    total_unused_size += size
    print(f"   • {img:<25} | {size_mb:>6.2f} MB")

print(f"\n🚫 MISSING IMAGES ({len(missing_images)}):")
for img in missing_images:
    print(f"   • {img} (referenced in code but not found in assets)")

print(f"\n💾 STORAGE IMPACT")
print("=" * 60)
total_assets_size = sum(assets_images.values())
unused_size_mb = total_unused_size / (1024 * 1024)
total_size_mb = total_assets_size / (1024 * 1024)
savings_percent = (total_unused_size / total_assets_size) * 100

print(f"Total assets size: {total_size_mb:.2f} MB")
print(f"Unused images size: {unused_size_mb:.2f} MB")
print(f"Potential savings: {savings_percent:.1f}%")

print(f"\n🛠️  CLEANUP RECOMMENDATIONS")
print("=" * 60)

print("🗑️  SAFE TO DELETE:")
for img, size in unused_images:
    size_mb = size / (1024 * 1024)
    print(f"   • {img:<25} | {size_mb:>6.2f} MB | Reason: Not referenced in code")

print(f"\n⚠️  DUPLICATE RESOLUTION:")
print("   Car Loan Images - Choose ONE to keep:")
print("   • Keep: Carloan.png (1.12 MB) - Currently referenced as 'carloan.jpg'")
print("   • Delete: Carloan.jpg (2.09 MB) - Larger file, not directly used")
print("   • Delete: carloan.jpg (0.10 MB) - Smaller version, not used")
print("   • Action: Update code to reference 'Carloan.png' instead of 'carloan.jpg'")

print(f"\n📋 CLEANUP COMMANDS")
print("=" * 60)
print("Run these commands to clean up duplicate and unused images:")
print()

# Generate cleanup commands
cleanup_commands = []
for img, size in unused_images:
    if img not in ['Carloan.jpg', 'carloan.jpg']:  # Handle car loan separately
        cleanup_commands.append(f"rm /workspace/shadcn-ui/public/assets/{img}")

# Add car loan cleanup
cleanup_commands.extend([
    "# Car loan cleanup - keep Carloan.png, remove duplicates",
    "rm /workspace/shadcn-ui/public/assets/Carloan.jpg",
    "rm /workspace/shadcn-ui/public/assets/carloan.jpg"
])

for cmd in cleanup_commands:
    if cmd.startswith('#'):
        print(f"\n{cmd}")
    else:
        print(f"   {cmd}")

print(f"\n📈 SUMMARY")
print("=" * 60)
print(f"• Total images in assets: {len(assets_images)}")
print(f"• Images currently used: {len(used_images)}")
print(f"• Images safe to delete: {len(unused_images)}")
print(f"• Missing images (need to add): {len(missing_images)}")
print(f"• Duplicate car loan files: 3 (can reduce to 1)")
print(f"• Storage savings: {unused_size_mb:.2f} MB ({savings_percent:.1f}%)")

print(f"\n✅ NEXT STEPS")
print("=" * 60)
print("1. Delete unused images using the commands above")
print("2. Update code references to use consistent file names")
print("3. Add missing testimonial images to assets directory")
print("4. Consider optimizing remaining images for web (compress/resize)")

=== COMPREHENSIVE IMAGE CLEANUP REPORT ===

🔍 DUPLICATE IMAGE ANALYSIS
🚗 Car Loan Image Duplicates Found:
   • Carloan.jpg          | 2.09 MB | ✅ USED
   • carloan.jpg          | 0.09 MB | ✅ USED
   • carloan.png          | 1.12 MB | ❌ UNUSED

📊 USAGE ANALYSIS
✅ IMAGES IN USE (9):
   • businessloan.png          |   0.99 MB
   • Carloan.jpg               |   2.09 MB
   • carloan.jpg               |   0.09 MB
   • herobanner.png            |   2.04 MB
   • homeloan.png              |   1.06 MB
   • loanagainstproperty.png   |   1.19 MB
   • logo-alt.png              |   0.70 MB
   • logo.png                  |   0.28 MB
   • personalloan.png          |   1.16 MB

❌ UNUSED IMAGES (5):
   • carloan.png               |   1.12 MB
   • favicon.ico               |   0.01 MB
   • instagram.svg             |   0.00 MB
   • whatsapp.svg              |   0.00 MB
   • youtube.svg               |   0.00 MB

🚫 MISSING IMAGES (5):
   • testimonial-2.jpg (referenced in code but not found in assets)
   